## Tuning NosAdam on MNIST

# 1.Imports & environment

Loading the NosAdam optimizer, source: https://github.com/andrehuang/NostalgicAdam-NosAdam 

In [ ]:
import math
import torch
from torch.optim.optimizer import Optimizer
import numpy as np


class NosAdam(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8,
                 weight_decay=0, gamma=0, lr_decay=False):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
        if not 0.0 <= gamma:
            raise ValueError("Invalid gamma value: {}".format(gamma))
        defaults = dict(lr=lr, betas=betas, eps=eps,
                        weight_decay=weight_decay, gamma=gamma, lr_decay=lr_decay)
        super(NosAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(NosAdam, self).__setstate__(state)
        for group in self.param_groups:
            group.setdefault('lr_decay', False)

    def step(self, closure=None):
        """Performs a single optimization step.

        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError('Adam does not support sparse gradients, please consider SparseAdam instead')
                # amsgrad = group['amsgrad']

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    # Exponential moving average of gradient values
                    state['exp_avg'] = torch.zeros_like(p.data)
                    # Exponential moving average of squared gradient values
                    state['exp_avg_sq'] = torch.zeros_like(p.data)

                    state['B_old'] = 0
                    state['B_new'] = 1
                    # if amsgrad:
                        # Maintains max of all exp. moving avg. of sq. grad. values
                        # state['max_exp_avg_sq'] = torch.zeros_like(p.data)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                # if amsgrad:
                #     max_exp_avg_sq = state['max_exp_avg_sq']
                beta1, beta2 = group['betas']
                beta2 = state['B_old']/state['B_new']
                gamma = group['gamma']
                # pnorm = group['pnorm']
                lr_decay = group['lr_decay']

                state['step'] += 1

                step = state['step']
                state['B_old'] += math.pow(step, -gamma)
                state['B_new'] += math.pow(step+1, -gamma)


                if group['weight_decay'] != 0:
                    grad = grad.add(group['weight_decay'], p.data)

                # Decay the first and second moment running average coefficient
                exp_avg.mul_(beta1).add_(1 - beta1, grad)
                # if pnorm == 2:
                #     exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                # elif pnorm ==3:
                #     # exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, np.multiply(grad, grad))
                #     exp_avg_sq = beta2*exp_avg_sq + (1 - beta2) * np.power(grad, 3)
                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)

                # if amsgrad:
                #     # Maintains the maximum of all 2nd moment running avg. till now
                #     torch.max(max_exp_avg_sq, exp_avg_sq, out=max_exp_avg_sq)
                #     # Use the max. for normalizing running avg. of gradient
                #     denom = max_exp_avg_sq.sqrt().add_(group['eps'])
                # else:
                # if pnorm ==2:
                #     denom = exp_avg_sq.sqrt().add_(group['eps'])
                # elif pnorm==3: # pnorm = 3
                #     denom = np.cbrt(exp_avg_sq) + group['eps']

                denom = exp_avg_sq.sqrt().add_(group['eps'])
                bias_correction1 = 1 - beta1 ** state['step']
                # AdaStab no longer needs bias correction for v_t
                # bias_correction2 = 1 - beta2 ** state['step']
                step_size = group['lr'] / bias_correction1
                if lr_decay:
                    step_size = step_size/math.sqrt(state['step'])


                p.data.addcdiv_(-step_size, exp_avg, denom)

        return loss

    def denominator(self):
        denom = np.array([0])
        denom_sum = 0
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError('Adam does not support sparse gradients, please consider SparseAdam instead')

                state = self.state[p]
                exp_avg_sq = state["exp_avg_sq"]

                exp_avg_sq = exp_avg_sq.numpy()
                exp_avg_sq = np.reshape(exp_avg_sq, -1)
                denom = np.concatenate((denom, exp_avg_sq))
                denom_sum += torch.sum(exp_avg_sq)
        return denom, denom_sum

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os
import gzip
import numpy as np
import sys
# os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# Setup predictable randomization
seed = 10
np.random.seed(seed)

# Setup CUda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. Loading and preparimg the data
As a basis for comparison we will be using the MNIST dataset. If we manage to do all the work we want, we will then use other datasets for comparison.

### 2.1. Definition of methods to extract data and labels

 Source: Image Analysis and Pattern Recognition (EE-451) lab 3

In [ ]:
def extract_data(filename, image_shape, image_number):
    with gzip.open(filename) as bytestream:
        bytestream.read(16)
        buf = bytestream.read(np.prod(image_shape) * image_number)
        data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
        data = data.reshape(image_number, image_shape[0], image_shape[1])
    return data


def extract_labels(filename, image_number):
    with gzip.open(filename) as bytestream:
        bytestream.read(8)
        buf = bytestream.read(1 * image_number)
        labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
    return labels

### 2.2. Load data

In [ ]:
image_shape = (28, 28)
train_set_size = 60000
test_set_size = 10000
data_folder = 'drive/MyDrive/mnist_data'

train_images_path = os.path.join(data_folder, 'train-images-idx3-ubyte.gz')
train_labels_path = os.path.join(data_folder, 'train-labels-idx1-ubyte.gz')
test_images_path = os.path.join(data_folder, 't10k-images-idx3-ubyte.gz')
test_labels_path = os.path.join(data_folder, 't10k-labels-idx1-ubyte.gz')

train_images = extract_data(train_images_path, image_shape, train_set_size)
test_images = extract_data(test_images_path, image_shape, test_set_size)
train_labels = extract_labels(train_labels_path, train_set_size)
test_labels = extract_labels(test_labels_path, test_set_size)

### 2.3. Convert data from numpy arrays to torch tensors

In [ ]:
features_train=torch.from_numpy(train_images).to(device)
features_test=torch.from_numpy(test_images).to(device)
print('Training features:', features_train.shape, '\n'
'Testing features:', features_test.shape)

labels_train=torch.from_numpy(train_labels).to(device)
labels_test=torch.from_numpy(test_labels).to(device)
print('Training labels:', labels_train.shape, '\n'
'Testing labels:', labels_test.shape)

Training features: torch.Size([60000, 28, 28]) 
Testing features: torch.Size([10000, 28, 28])
Training labels: torch.Size([60000]) 
Testing labels: torch.Size([10000])


### 2.4. Normalize data

In [ ]:
mean, std = features_train.float().mean(), features_train.float().std()

features_train = features_train.float().sub_(mean).div_(std)
features_test = features_test.float().sub_(mean).div_(std)

In [ ]:
#reshape to make the 1st channel be batch size

features_train = features_train.reshape(-1, 1, 28, 28)
features_test = features_test.reshape(-1, 1, 28, 28)

# 3. Setting up networks and evaluation methods

### 3.1. Multilayer perceptron (MLP)

In [ ]:
class MLP(nn.Module):
    
    def __init__(self, hidden_size_1=512, hidden_size_2=100, hidden_size_3=10):
        super(MLP, self).__init__()
        
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784, hidden_size_1),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_size_1, hidden_size_2),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_size_2, hidden_size_3))
    
    def forward(self, x):
        return self.layers(x)

###3.2. Convolutional Neural Network (CNN)

In [ ]:
from numpy.ma.core import reshape
class CNN(nn.Module):
   def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 32, kernel_size=5, stride=1, padding='same')
        self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=5, stride=1, padding='same')
        self.fc1 = nn.Linear(64*7*7, 1024)
        self.fc2 = nn.Linear(1024, 10)

   def forward(self, input:torch.Tensor) -> torch.Tensor:
        pool1 = torch.max_pool2d(F.relu(self.conv1(input)), kernel_size=2, stride=2)
        pool2 = torch.max_pool2d(F.relu(self.conv2(pool1)), kernel_size=2, stride=2)
        res = pool2.reshape(-1, 64*7*7)
        hidden = F.relu(self.fc1(res))
        output = self.fc2(hidden)

        return output

### 3.3. Implementation of method for training

In [ ]:
def run_nn(x_train, y_train, x_test, y_test, model, optimizer, criterion, num_epoch, size_minibatch):

    loss_train_ret = 0
    loss_test_ret = 0
    loss_train = 0
            
    for epoch in range(num_epoch):
        for b in range(0, x_train.size(0), size_minibatch):
            y = model(x_train[b:b+size_minibatch])
            loss_train = criterion(y, y_train[b:b+size_minibatch])
        
            optimizer.zero_grad()
            loss_train.backward()
            optimizer.step()

        if epoch == num_epoch - 1:

            y_test_obt = model(x_test)
            loss_test = criterion(y_test_obt, y_test)
            
            loss_train_ret = loss_train
            loss_test_ret = loss_test
            
            print('Final, Train Loss: %.4f, Test Loss: %.4f' %(loss_train, loss_test))

    return loss_train_ret, loss_test_ret

# 4. Metrics of our tuning protocol
At this stage, we want to select the hyperparameter search space for each optimizer. This way, we can first tune the hyperparameters of each optimizer separately and then select the trial that achieved lowest final validation error.
We then comapre the optimizers' performance by looking at the validation and test errors as suggested in the paper "On empirical comparisons of optimizers for deep learning".

No regularization decay is used.

### 4.1. Tuning protocol using bootstrap
To estimate means and uncertainties of our tuning protocol we will use bootstrapping starting from an initial search space suggested by the paper "On Empirical Comparisons of Optimizers for Deep Learning".
We run N trials by randomly picking values in the search space of the algorithm at every trial.
Then we sample these trials with replacement and compute our statistic on the first K trials of this sample. We repeat this process 50 or 100 times and compute the 5th percentile and 95th percentile of the bootstrap distribution.

This allows us to plot the error bars to show the results.

### 4.2. Tuning NosAdam for the MLP on MNIST
The hyperparameters we are tuning are alpha_0/epsilon, 1 - beta_1, 1 - beta_2, epsilon.
The initial search spaces are suggested based on the experience of the writers of the same paper, "On empirical comparisons of optimizers for deep learning".

##### Set up model for training

In [ ]:
# Model fixed parameters
model = MLP()
model = model.to(device)
criterion = nn.CrossEntropyLoss().to(device) # good loss function for classification tasks
num_epoch = 50
size_minibatch = 128

x_train = features_train
y_train = labels_train
x_test = features_test
y_test = labels_test

##### Tune to find best parameter
We perform trials until we have K of them, then we pick the best based on our statistic of interest

#### 4.2.1. Initial search for best hyperparameters for **NosAdam** optimizer on **MLP**. K= 100. 
Interrupted because of nan loss.

##### Set up parameters and search space for the initial trial

In [ ]:
N = 200
K = 100 # Number of trials being kept for the statistic

# Initial search spaces for parameters
alpha_0 = np.linspace(10**(-2), 10**(4), N)
beta_1 = np.linspace(10**(-3), 1, N)
beta_2 = np.linspace(10**(-4), 1, N)
eps = np.linspace(10**(-10), 10**(10), N)

Perform search

In [ ]:
nb_hyperamaters_to_tune = 4
nb_exported_statistics  = 2

lowest_test_error = [sys.maxsize] * (nb_hyperamaters_to_tune + nb_exported_statistics)

for _ in range(K):
    # Pick random values from the intervals given for the different parameters
    alpha_0_pick  = float(np.random.choice(alpha_0, 1)) # np.random.choice samples uniformely with replacement
    beta_1_pick   = float(-np.random.choice(beta_1, 1) + 1)
    beta_2_pick   = float(-np.random.choice(beta_2, 1) + 1)
    eps_pick      = float(np.random.choice(eps, 1))
    learning_rate = alpha_0_pick * eps_pick
    
    # Build optimizer from parameters
    model=MLP()
    model=model.to(device)
    optimizer = NosAdam(model.parameters(), lr=learning_rate, betas=(beta_1_pick, beta_2_pick), eps=eps_pick)
    
    # Run
    train_error, test_error = run_nn(x_train,y_train, x_test, y_test, model, optimizer, criterion, num_epoch, size_minibatch)
    
    # Concatenate hyperparameters with results
    vector = [beta_1_pick, beta_2_pick, eps_pick, learning_rate, train_error, test_error]
    
    # Check wether we have the smallest test error and store parameters in case we find it
    if test_error < lowest_test_error[len(lowest_test_error) - 1]:
        lowest_test_error = vector

c:\Users\Claudia Naumova\Opt_ML\OptML_project\NosAdam.py:86: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:1050.)
  exp_avg.mul_(beta1).add_(1 - beta1, grad)


Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: 0.5216, Test Loss: 0.4796
Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: nan, Test Loss: nan


KeyboardInterrupt: 

#### 4.2.2. Final search for best hyperparameters for **NosAdam** optimizer on **MLP**. K = 50

##### Set up parameters and search space for the final trial

In [ ]:
N = 200
K = 50 # Number of trials being kept for the statistic

# Final search spaces for parameters
alpha_0 = np.linspace(10**(-1), 10, N)
beta_1 = np.linspace(10**(-3), 1, N)
beta_2 = np.linspace(10**(-4), 1, N)
eps = np.linspace(10**(-6), 10**(-2), N)

Perform search

In [ ]:
nb_hyperamaters_to_tune = 4
nb_exported_statistics  = 2

lowest_test_error = [sys.maxsize] * (nb_hyperamaters_to_tune + nb_exported_statistics)


for _ in range(K):
    # Pick random values from the intervals given for the different parameters
    alpha_0_pick  = float(np.random.choice(alpha_0, 1)) # np.random.choice samples uniformely with replacement
    beta_1_pick   = float(-np.random.choice(beta_1, 1) + 1)
    beta_2_pick   = float(-np.random.choice(beta_2, 1) + 1)
    eps_pick      = float(np.random.choice(eps, 1))
    learning_rate = alpha_0_pick * eps_pick
    
    # Build optimizer from parameters
    model = MLP()
    model = model.to(device)
    optimizer = NosAdam(model.parameters(), lr=learning_rate, betas=(beta_1_pick, beta_2_pick), eps=eps_pick)
    
    # Run
    train_error, test_error = run_nn(x_train,y_train, x_test, y_test, model, optimizer, criterion, num_epoch, size_minibatch)
    
    # Concatenate hyperparameters with results
    vector = [beta_1_pick, beta_2_pick, eps_pick, learning_rate, train_error, test_error]
    
    # Check wether we have the smallest test error and store parameters in case we find it
    if test_error < lowest_test_error[len(lowest_test_error) - 1]:
        lowest_test_error = vector

c:\Users\Claudia Naumova\Opt_ML\OptML_project\NosAdam.py:86: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:1050.)
  exp_avg.mul_(beta1).add_(1 - beta1, grad)


Final, Train Loss: 0.0001, Test Loss: 0.0810
Final, Train Loss: 1.8264, Test Loss: 1.8137
Final, Train Loss: 0.2071, Test Loss: 0.1658
Final, Train Loss: 0.0000, Test Loss: 0.0994
Final, Train Loss: 2.3058, Test Loss: 2.3015
Final, Train Loss: 0.0387, Test Loss: 0.2560
Final, Train Loss: 0.2790, Test Loss: 0.4581
Final, Train Loss: 1.7682, Test Loss: 1.7026
Final, Train Loss: 2.3060, Test Loss: 2.3016
Final, Train Loss: 0.0476, Test Loss: 0.2908
Final, Train Loss: 0.1852, Test Loss: 0.3003
Final, Train Loss: 0.0000, Test Loss: 0.1531
Final, Train Loss: 2.3071, Test Loss: 2.3018
Final, Train Loss: 0.0236, Test Loss: 0.3225
Final, Train Loss: 0.0000, Test Loss: 0.1110
Final, Train Loss: 2.3067, Test Loss: 2.3018
Final, Train Loss: 0.0000, Test Loss: 0.1217
Final, Train Loss: 2.3075, Test Loss: 2.3027
Final, Train Loss: 2.3052, Test Loss: 2.3016
Final, Train Loss: 0.0028, Test Loss: 0.2247
Final, Train Loss: 1.8098, Test Loss: 1.8529
Final, Train Loss: 0.0000, Test Loss: 0.0846
Final, Tra

In [ ]:
# Print best parameters

print('Beta 1: %.2f' % lowest_test_error[0])
print('Beta 2: %.2f' % lowest_test_error[1])
print('Epsilon: %.2e' % lowest_test_error[2])
print('Learning rate: %.6f' % lowest_test_error[3])
print('Train error: %.6f' % lowest_test_error[4])
print('Test error: %.4f' % lowest_test_error[5])

Beta 1: 0.25
Beta 2: 0.84
Epsilon: 1.52e-04
Learning rate: 0.000068
Train error: 0.006410
Test error: 0.0652


Run the model again with the best parameters to plot the convergence later

In [ ]:
model = MLP()
model = model.to(device)
learning_rate = lowest_test_error[3]
beta_1 = lowest_test_error[0]
beta_2 = lowest_test_error[1]
eps = lowest_test_error[2]
optimizer = NosAdam(model.parameters(), lr=learning_rate, betas=(beta_1, beta_2), eps=eps)
    
loss_all_train_mlp, loss_all_test_mlp = [], []
            
for epoch in range(num_epoch):
     for b in range(0, x_train.size(0), size_minibatch):
            y = model(x_train[b:b+size_minibatch])
            loss_train = criterion(y, y_train[b:b+size_minibatch])
        
            optimizer.zero_grad()
            loss_train.backward()
            optimizer.step()

     loss_train = loss_train.to('cpu').detach().numpy()
     loss_all_train_mlp.append(loss_train)

     y_test_obt = model(x_test)
     loss_test = criterion(y_test_obt, y_test)
     loss_test = loss_test.to('cpu').detach().numpy()
     loss_all_test_mlp.append(loss_test)
     if epoch == num_epoch - 1 :
        print('Final, Train Loss: %.4f, Test Loss: %.4f' %(loss_train, loss_test))

Final, Train Loss: 0.0079, Test Loss: 0.0634


In [ ]:
#save the results

import pickle
with open("plot_nosadam_mlp_train.pth", "wb") as fp:
  pickle.dump(loss_all_train_mlp, fp)

with open("plot_nosadam_mlp_test.pth", "wb") as fp:
  pickle.dump(loss_all_test_mlp, fp)

##### 4.2.3. Estimating trial outcomes via bootstrap
At this stage we want to estimate means and uncertainties of our tuning protocol

###### Run N trials

In [ ]:
# We first run and store N trials
N=200
N_trials = []


for _ in range(N):
    # Pick random values from the intervals given for the different parameters
    alpha_0_pick  = float(np.random.choice(alpha_0, 1)) # np.random.choice samples uniformely with replacement
    beta_1_pick   = float(-np.random.choice(beta_1, 1) + 1)
    beta_2_pick   = float(-np.random.choice(beta_2, 1) + 1)
    eps_pick      = float(np.random.choice(eps, 1))
    learning_rate = alpha_0_pick * eps_pick
    
    # Build optimizer from parameters
    model = MLP()
    model = model.to(device)
    optimizer = NosAdam(model.parameters(), lr=learning_rate, betas=(beta_1_pick, beta_2_pick), eps=eps_pick)
    
    # Run
    train_error, test_error = run_nn(x_train,y_train, x_test, y_test, model, optimizer, criterion, num_epoch, size_minibatch)
    
    # Store parameters, train and test error
    N_trials.append([beta_1_pick, beta_2_pick, eps_pick, learning_rate, train_error.item(), test_error.item()])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:86: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1055.)


Final, Train Loss: 1.8994, Test Loss: 1.7757
Final, Train Loss: 0.2170, Test Loss: 0.1633
Final, Train Loss: 0.0000, Test Loss: 0.0932
Final, Train Loss: 2.3058, Test Loss: 2.3016
Final, Train Loss: 0.0212, Test Loss: 0.2788
Final, Train Loss: 1.6695, Test Loss: 1.4105
Final, Train Loss: 0.9822, Test Loss: 0.9878
Final, Train Loss: 1.6502, Test Loss: 1.6149
Final, Train Loss: 0.5107, Test Loss: 0.3931
Final, Train Loss: 0.0437, Test Loss: 0.2538
Final, Train Loss: 0.0001, Test Loss: 0.1491
Final, Train Loss: 2.0552, Test Loss: 1.9812
Final, Train Loss: 0.0766, Test Loss: 0.2807
Final, Train Loss: 0.0000, Test Loss: 0.1039
Final, Train Loss: 2.2462, Test Loss: 2.0222
Final, Train Loss: 0.0001, Test Loss: 0.1657
Final, Train Loss: 2.3075, Test Loss: 2.3027
Final, Train Loss: 2.3052, Test Loss: 2.3017
Final, Train Loss: 0.0501, Test Loss: 0.3256
Final, Train Loss: 2.3072, Test Loss: 2.3017
Final, Train Loss: 0.0000, Test Loss: 0.0926
Final, Train Loss: 2.3053, Test Loss: 2.3015
Final, Tra

In [ ]:
#save the results

import pickle
with open("N_trials_nosadam_mlp.pth", "wb") as fp:
  pickle.dump(N_trials, fp)

### 4.3. Tuning NosAdam for the CNN on MNIST

##### Set up model for training

In [ ]:
# Model fixed parameters
model = CNN()
model = model.to(device)
criterion = nn.CrossEntropyLoss().to(device) # good loss function for classification tasks
num_epoch = 50
size_minibatch = 128

x_train = features_train
y_train = labels_train
x_test = features_test
y_test = labels_test

##### Tune to find best parameter
We perform trials until we have K of them, then we pick the best based on our statistic of interest

#### 4.3.1. Initial search for best hyperparameters for **NosAdam** optimizer on **CNN**. K= 100. 
Interrupted because of nan loss.

##### Set up parameters and search space for the initial trial

In [ ]:
N = 200
K = 100 # Number of trials being kept for the statistic

# Initial search spaces for parameters
alpha_0 = np.linspace(10**(-2), 10**(4), N)
beta_1 = np.linspace(10**(-3), 1, N)
beta_2 = np.linspace(10**(-4), 1, N)
eps = np.linspace(10**(-10), 10**(10), N)

Perform search

In [ ]:
nb_hyperamaters_to_tune = 4
nb_exported_statistics  = 2

lowest_test_error = [sys.maxsize] * (nb_hyperamaters_to_tune + nb_exported_statistics)

for _ in range(K):
    # Pick random values from the intervals given for the different parameters
    alpha_0_pick  = float(np.random.choice(alpha_0, 1)) # np.random.choice samples uniformely with replacement
    beta_1_pick   = float(-np.random.choice(beta_1, 1) + 1)
    beta_2_pick   = float(-np.random.choice(beta_2, 1) + 1)
    eps_pick      = float(np.random.choice(eps, 1))
    learning_rate = alpha_0_pick * eps_pick
    
    # Build optimizer from parameters
    model=CNN()
    model=model.to(device)
    optimizer = NosAdam(model.parameters(), lr=learning_rate, betas=(beta_1_pick, beta_2_pick), eps=eps_pick)
    
    # Run
    train_error, test_error = run_nn(x_train,y_train, x_test, y_test, model, optimizer, criterion, num_epoch, size_minibatch)
    # train_error, test_error = run_nn(x_train,y_train, x_test, y_test, model, optimizer, criterion, num_epoch)
    
    
    # Concatenate hyperparameters with results
    vector = [beta_1_pick, beta_2_pick, eps_pick, learning_rate, train_error, test_error]
    
    # Check wether we have the smallest test error and store parameters in case we find it
    if test_error < lowest_test_error[len(lowest_test_error) - 1]:
        lowest_test_error = vector

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:86: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1055.)


Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: 0.2715, Test Loss: 0.1980
Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: 1954.3563, Test Loss: 2359.0366
Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: nan, Test Loss: nan
Final, Train Loss: 1335.0944, Test Loss: 865.0282
Final, Train Loss: nan, Test Loss: nan


KeyboardInterrupt: ignored

#### 4.3.2. Final search for best hyperparameters for **NosAdam** optimizer on **CNN**. K = 50

##### Set up parameters and search space for the final trial

In [ ]:
N = 200
K = 50 # Number of trials being kept for the statistic

# Final search spaces for parameters
alpha_0 = np.linspace(10**(-1), 10, N)
beta_1 = np.linspace(10**(-3), 1, N)
beta_2 = np.linspace(10**(-4), 1, N)
eps = np.linspace(10**(-6), 10**(-2), N)

Perform search

In [ ]:
nb_hyperamaters_to_tune = 4
nb_exported_statistics  = 2

lowest_test_error = [sys.maxsize] * (nb_hyperamaters_to_tune + nb_exported_statistics)


for _ in range(K):
    # Pick random values from the intervals given for the different parameters
    alpha_0_pick  = float(np.random.choice(alpha_0, 1)) # np.random.choice samples uniformely with replacement
    beta_1_pick   = float(-np.random.choice(beta_1, 1) + 1)
    beta_2_pick   = float(-np.random.choice(beta_2, 1) + 1)
    eps_pick      = float(np.random.choice(eps, 1))
    learning_rate = alpha_0_pick * eps_pick
    
    # Build optimizer from parameters
    model=CNN()
    model=model.to(device)
    optimizer = NosAdam(model.parameters(), lr=learning_rate, betas=(beta_1_pick, beta_2_pick), eps=eps_pick)
    
    # Run
    train_error, test_error = run_nn(x_train,y_train, x_test, y_test, model, optimizer, criterion, num_epoch, size_minibatch)
    
    # Concatenate hyperparameters with results
    vector = [beta_1_pick, beta_2_pick, eps_pick, learning_rate, train_error, test_error]
    
    # Check wether we have the smallest test error and store parameters in case we find it
    if test_error < lowest_test_error[len(lowest_test_error) - 1]:
        lowest_test_error = vector

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:86: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1055.)


Final, Train Loss: 0.0000, Test Loss: 0.0367
Final, Train Loss: 2.3064, Test Loss: 2.3018
Final, Train Loss: 0.1897, Test Loss: 0.0373
Final, Train Loss: 0.0000, Test Loss: 0.0325
Final, Train Loss: 2.3057, Test Loss: 2.3015
Final, Train Loss: 0.1970, Test Loss: 0.2692
Final, Train Loss: 2.3053, Test Loss: 2.3015
Final, Train Loss: 2.3062, Test Loss: 2.3015
Final, Train Loss: 2.3060, Test Loss: 2.3016
Final, Train Loss: 2.3049, Test Loss: 2.3015
Final, Train Loss: 0.0000, Test Loss: 0.1537
Final, Train Loss: 0.0852, Test Loss: 0.1191
Final, Train Loss: 2.3072, Test Loss: 2.3018
Final, Train Loss: 2.3044, Test Loss: 2.3015
Final, Train Loss: 0.0000, Test Loss: 0.0440
Final, Train Loss: 2.3069, Test Loss: 2.3018
Final, Train Loss: 0.0000, Test Loss: 0.0373
Final, Train Loss: 2.3057, Test Loss: 2.3021
Final, Train Loss: 2.3053, Test Loss: 2.3016
Final, Train Loss: 0.5275, Test Loss: 0.4310
Final, Train Loss: 2.3070, Test Loss: 2.3019
Final, Train Loss: 0.0000, Test Loss: 0.0371
Final, Tra

In [ ]:
# Print best parameters

print('Beta 1: %.2f' % lowest_test_error[0])
print('Beta 2: %.2f' % lowest_test_error[1])
print('Epsilon: %.2e' % lowest_test_error[2])
print('Learning rate: %.2e' % lowest_test_error[3])
print('Train error: %.6f' % lowest_test_error[4])
print('Test error: %.4f' % lowest_test_error[5])

Beta 1: 0.18
Beta 2: 0.42
Epsilon: 8.05e-04
Learning rate: 1.68e-03
Train error: 0.000009
Test error: 0.0325


Run the model again with the best parameters to plot the convergence later

In [ ]:
model = CNN()
model = model.to(device)
learning_rate = lowest_test_error[3]
beta_1 = lowest_test_error[0]
beta_2 = lowest_test_error[1]
eps = lowest_test_error[2]
optimizer = NosAdam(model.parameters(), lr=learning_rate, betas=(beta_1, beta_2), eps=eps)
    
loss_all_train_cnn, loss_all_test_cnn = [], []
            
for epoch in range(num_epoch):
     for b in range(0, x_train.size(0), size_minibatch):
            y = model(x_train[b:b+size_minibatch])
            loss_train = criterion(y, y_train[b:b+size_minibatch])
        
            optimizer.zero_grad()
            loss_train.backward()
            optimizer.step()

     loss_train = loss_train.to('cpu').detach().numpy()
     loss_all_train_cnn.append(loss_train)

     y_test_obt = model(x_test)
     loss_test = criterion(y_test_obt, y_test)
     loss_test = loss_test.to('cpu').detach().numpy()
     loss_all_test_cnn.append(loss_test)
     if epoch == num_epoch - 1 :
        print('Final, Train Loss: %.4f, Test Loss: %.4f' %(loss_train, loss_test))

Final, Train Loss: 0.0005, Test Loss: 0.0333


In [ ]:
#save the results

import pickle
with open("plot_nosadam_cnn_train.pth", "wb") as fp:
  pickle.dump(loss_all_train_cnn, fp)

with open("plot_nosadam_cnn_test.pth", "wb") as fp:
  pickle.dump(loss_all_test_cnn, fp)

##### 4.3.3. Estimating trial outcomes via bootstrap
At this stage we want to estimate means and uncertainties of our tuning protocol

###### Run N trials

In [ ]:
N = 50
K = 25 # Number of trials being kept for the statistic

# Final search spaces for parameters
alpha_0 = np.linspace(10**(-1), 10, N)
beta_1 = np.linspace(10**(-3), 1, N)
beta_2 = np.linspace(10**(-4), 1, N)
eps = np.linspace(10**(-6), 10**(-2), N)

In [ ]:
# We first run and store N trials
N_trials = []
model = CNN()
model = model.to(device)

for _ in range(N):
    # Pick random values from the intervals given for the different parameters
    alpha_0_pick  = float(np.random.choice(alpha_0, 1)) # np.random.choice samples uniformely with replacement
    beta_1_pick   = float(-np.random.choice(beta_1, 1) + 1)
    beta_2_pick   = float(-np.random.choice(beta_2, 1) + 1)
    eps_pick      = float(np.random.choice(eps, 1))
    learning_rate = alpha_0_pick * eps_pick
    
    # Build optimizer from parameters
    model = CNN()
    model = model.to(device)
    optimizer = NosAdam(model.parameters(), lr=learning_rate, betas=(beta_1_pick, beta_2_pick), eps=eps_pick)
    
    # Run
    train_error, test_error = run_nn(x_train,y_train, x_test, y_test, model, optimizer, criterion, num_epoch, size_minibatch)
    
    # Store parameters, train and test error
    N_trials.append([beta_1_pick, beta_2_pick, eps_pick, learning_rate, train_error.item(), test_error.item()])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:86: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1055.)


Final, Train Loss: 0.2019, Test Loss: 0.0812
Final, Train Loss: 2.3071, Test Loss: 2.3020
Final, Train Loss: 2.3053, Test Loss: 2.3015
Final, Train Loss: 0.0392, Test Loss: 0.1599
Final, Train Loss: 2.3057, Test Loss: 2.3023
Final, Train Loss: 2.3054, Test Loss: 2.3015
Final, Train Loss: 2.3052, Test Loss: 2.3015
Final, Train Loss: 0.0004, Test Loss: 0.2112
Final, Train Loss: 2.3052, Test Loss: 2.3015
Final, Train Loss: 0.0000, Test Loss: 0.0382
Final, Train Loss: 2.3058, Test Loss: 2.3015
Final, Train Loss: 2.3053, Test Loss: 2.3015
Final, Train Loss: 2.3050, Test Loss: 2.3015
Final, Train Loss: 2.3055, Test Loss: 2.3014
Final, Train Loss: 0.0799, Test Loss: 0.2846
Final, Train Loss: 2.3049, Test Loss: 2.3015
Final, Train Loss: 2.3063, Test Loss: 2.3017
Final, Train Loss: 2.3043, Test Loss: 2.3014
Final, Train Loss: 0.0019, Test Loss: 0.3783
Final, Train Loss: 2.3057, Test Loss: 2.3015
Final, Train Loss: 2.3053, Test Loss: 2.3015
Final, Train Loss: 2.3071, Test Loss: 2.3024
Final, Tra

In [ ]:
#save the results

import pickle
with open("N_trials_nosadam_cnn.pth", "wb") as fp:
  pickle.dump(N_trials, fp)